# XGboost이용한 유사도 판별
앙상블 모델 중 하나.

XG 부스트란 eXtream Gradient Boosting의 약자로 최근 캐글 사용자에게 큰 인기를 
얻고 있는 모델 중 하나. 

- 앙상블 : 머신러닝 혹은 통계학에서 여러개의 학습 알고리즘을 사용해 더 좋은 성능을 얻는 방법을 뜻한다.

    - 배깅 : 여러개의 학습 알고리즘, 모델을 통해 각각 결과를 예측하고 모든 결과를 동등하게 보고 취합하는 방식 ( ex 랜덤 포레스트)

    - 부스팅 : 각 결과를 순차적으로 취합. 이전 알고리즘, 모델이 학습 후 잘못 예측한 부분에 가중치를 줘서 다시 모델로 가서 학습하는 방법.

XG 부스트는 여러 tree모델을 부스팅 방식으로 앙상블 하여 사용하는 모델

랜덤 포레스트와 동일한 원리에서, 가중치를 부여한 것



In [ ]:
import pandas as pd
import numpy as np
import os

import json

from sklearn.model_selection  import train_test_split

In [ ]:
DATA_IN_PATH = '/content/drive/MyDrive/2021-1/AI데이터활용교재개발/자연어처리(텐서플로,머신러닝)/code/data/quora-question-pairs/'
DATA_OUT_PATH =  '/content/drive/MyDrive/2021-1/AI데이터활용교재개발/자연어처리(텐서플로,머신러닝)/code/data/quora-question-pairs/out/'


TRAIN_Q1_DATA_FILE = 'train_q1.npy'
TRAIN_Q2_DATA_FILE = 'train_q2.npy'
TRAIN_LABEL_DATA_FILE = 'train_label.npy'


# 훈련 데이터 가져오는 부분이다.
train_q1_data = np.load(open(DATA_IN_PATH + TRAIN_Q1_DATA_FILE, 'rb'))
train_q2_data = np.load(open(DATA_IN_PATH + TRAIN_Q2_DATA_FILE, 'rb'))
train_labels = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA_FILE, 'rb'))

In [ ]:
train_input = np.stack((train_q1_data, train_q2_data), axis=1)
print(train_input.shape)

(298526, 2, 31)


In [ ]:
#훈련셋과 평가 셋 나누기
train_input, eval_input, train_label, eval_label = train_test_split(train_input, train_labels, test_size=0.2, random_state=4242)

## 모델 생성

In [ ]:
import xgboost as xgb


In [ ]:
train_data = xgb.DMatrix(train_input.sum(axis=1), label=train_label) # 학습 데이터 읽어 오기
eval_data = xgb.DMatrix(eval_input.sum(axis=1), label=eval_label) # 평가 데이터 읽어 오기

data_list = [(train_data, 'train'), (eval_data, 'valid')]
params = {} # 인자를 통해 XGB모델에 넣어 주자 
params['objective'] = 'binary:logistic' # 로지스틱 예측을 통해서 
params['eval_metric'] = 'rmse' # root mean square error를 사용  

bst = xgb.train(params, train_data, num_boost_round = 1000, evals = data_list, early_stopping_rounds=10)

[0]	train-rmse:0.483874	valid-rmse:0.484085
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[1]	train-rmse:0.473579	valid-rmse:0.474065
[2]	train-rmse:0.466641	valid-rmse:0.467503
[3]	train-rmse:0.461957	valid-rmse:0.463014
[4]	train-rmse:0.458234	valid-rmse:0.459588
[5]	train-rmse:0.455505	valid-rmse:0.457095
[6]	train-rmse:0.452957	valid-rmse:0.454734
[7]	train-rmse:0.451264	valid-rmse:0.453203
[8]	train-rmse:0.44966	valid-rmse:0.451723
[9]	train-rmse:0.448298	valid-rmse:0.450449
[10]	train-rmse:0.447134	valid-rmse:0.449467
[11]	train-rmse:0.444658	valid-rmse:0.44715
[12]	train-rmse:0.443646	valid-rmse:0.446368
[13]	train-rmse:0.443024	valid-rmse:0.445828
[14]	train-rmse:0.442316	valid-rmse:0.445256
[15]	train-rmse:0.44185	valid-rmse:0.444915
[16]	train-rmse:0.440294	valid-rmse:0.443496
[17]	train-rmse:0.439923	valid-rmse:0.443222
[18]	train-rmse:0.439381	valid-rmse:0.44288
[19]	train-rms

In [ ]:
TEST_Q1_DATA_FILE = 'test_q1.npy'
TEST_Q2_DATA_FILE = 'test_q2.npy'
TEST_ID_DATA_FILE = 'test_id.npy'

test_q1_data = np.load(open(DATA_IN_PATH + TEST_Q1_DATA_FILE, 'rb'))
test_q2_data = np.load(open(DATA_IN_PATH + TEST_Q2_DATA_FILE, 'rb'))
test_id_data = np.load(open(DATA_IN_PATH + TEST_ID_DATA_FILE, 'rb'))

In [ ]:
# XGboost를 학습시키기 위해서는 DMatrix라는 형태로 데이터를 바꿔줘야함
test_input = np.stack((test_q1_data, test_q2_data), axis=1) 
test_data = xgb.DMatrix(test_input.sum(axis=1))
test_predict = bst.predict(test_data)

In [ ]:
# 학습 후 test data에 대해 test, 후 Kaggle에 제출해보면 된다.
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
output = pd.DataFrame({'test_id': test_id_data, 'is_duplicate': test_predict})
output.to_csv(DATA_OUT_PATH + 'simple_xgb.csv', index=False)